In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt

In [74]:
# data: 
folder = 'data'
xls_dict  = pd.read_excel(folder + '/trading-game-data-20102023.xlsx', sheet_name=None)

index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

## Markowitz Portfolio Theory

In [75]:

price_df = xls_dict['price'].reset_index()
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.set_index('Date', inplace=True)
daily_returns = price_df.pct_change()
expected_returns = daily_returns.mean()
risk = daily_returns.std()

risk.sort_values()

WMT      0.008431
KO       0.008471
MCD      0.008852
BRK.B    0.008859
PG       0.009301
           ...   
SEDG     0.038979
CMA      0.040414
CTLT     0.041364
ZION     0.041840
index         NaN
Length: 501, dtype: float64

## CAPM model
- Time horizon is the full dataframe

In [62]:
def calc_CAPM_betas(daily_returns, sp_500_daily_returns):
    """
    Calculate the CAPM beta values for the stocks in the daily_returns DataFrame.
    """
    
    # Join the daily returns of the stocks with the S&P 500 daily returns
    daily_returns_with_sp500 = daily_returns.join(sp_500_daily_returns.rename('SP500'))
    
    # Calculate the covariance matrix of the returns
    cov_matrix_with_sp500 = daily_returns_with_sp500.cov()
    
    # The market variance is the variance of the S&P 500 returns
    market_var = sp_500_daily_returns.var()
    
    # Calculate the betas for each stock
    betas = cov_matrix_with_sp500.loc[:, 'SP500'] / market_var
    
    betas = betas.drop(['SP500', 'index'], axis=0)
    return betas

index_price_df = xls_dict['index-price'].reset_index()
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
sp_500_daily_returns = index_price_df['S&P 500'].pct_change()

betas = calc_CAPM_betas(daily_returns, sp_500_daily_returns)
betas

A       0.912992
AAL     1.348263
AAPL    1.150799
ABBV    0.130669
ABNB    1.690057
          ...   
YUM     0.556312
ZBH     0.571193
ZBRA    1.702181
ZION    2.221824
ZTS     0.977542
Name: SP500, Length: 500, dtype: float64

In [71]:
def calc_expectedreturns(daily_returns, rf_rate, betas, market_return):
    expected_returns = rf_rate + betas * (market_return - rf_rate)
    
    average_returns = daily_returns.mean() * 252  # Assuming 252 trading days in a year

    # Step 3: Determine undervalued/overvalued stocks
    comparison = pd.DataFrame({
        'Beta': betas,
        'Expected Return': expected_returns,
        'Average Return': average_returns
    })
    comparison['Over/Under Valued'] = comparison.apply(
        lambda row: 'Undervalued' if row['Average Return'] > row['Expected Return'] else 'Overvalued',
        axis=1
    )
    
    return comparison

if 'index' in daily_returns.columns:
    daily_returns = daily_returns.drop(['index'], axis=1).copy()

market_return = np.prod(1 + sp_500_daily_returns.dropna())**(252 / len(sp_500_daily_returns.dropna())) - 1
risk_free_rate = 0.0477 
result_df = calc_expectedreturns(daily_returns, risk_free_rate,betas, market_return)
print(result_df)

          Beta  Expected Return  Average Return Over/Under Valued
A     0.912992         0.119640       -0.361128        Overvalued
AAL   1.348263         0.153937       -0.107494        Overvalued
AAPL  1.150799         0.138378        0.378982       Undervalued
ABBV  0.130669         0.057996       -0.104357        Overvalued
ABNB  1.690057         0.180869        0.479042       Undervalued
...        ...              ...             ...               ...
YUM   0.556312         0.091535       -0.068445        Overvalued
ZBH   0.571193         0.092707       -0.225642        Overvalued
ZBRA  1.702181         0.181824       -0.195368        Overvalued
ZION  2.221824         0.222770       -0.390007        Overvalued
ZTS   0.977542         0.124726        0.193654       Undervalued

[500 rows x 4 columns]


In [72]:
undervalued_stocks = result_df[result_df['Over/Under Valued'] == 'Undervalued']
print(undervalued_stocks)

          Beta  Expected Return  Average Return Over/Under Valued
AAPL  1.150799         0.138378        0.378982       Undervalued
ABNB  1.690057         0.180869        0.479042       Undervalued
ACGL  0.669985         0.100492        0.374997       Undervalued
ACN   1.100715         0.134431        0.159935       Undervalued
ADBE  1.710577         0.182486        0.645633       Undervalued
...        ...              ...             ...               ...
WDC   1.241642         0.145536        0.452524       Undervalued
WELL  0.881319         0.117144        0.339109       Undervalued
WMT   0.386447         0.078150        0.149989       Undervalued
WST   0.823195         0.112564        0.583559       Undervalued
ZTS   0.977542         0.124726        0.193654       Undervalued

[159 rows x 4 columns]


In [87]:
data = pd.read_excel(folder + '/trading-game-data-20102023.xlsx', sheet_name='price')

n_stocks = 200
newdata = data.drop(data.columns[0], axis=1)
newdata = newdata.iloc[:,0:n_stocks]
returns = np.log(newdata/newdata.shift(1))
returns = returns.drop(returns.index[0])  

In [89]:
def objective_function(weights: list, returns):
    
    mean_returns = np.mean(returns, axis=0)
    portfolio_return = weights @  mean_returns

    portfolio_std = np.sqrt(weights @ np.cov(returns.T) @ weights.T)

    return -1 * (portfolio_return - 0.25 * portfolio_std)  # Minimize the negative of the objective

initial_weights = np.array([1 / n_stocks] * n_stocks)
constraints = ( 
        {'type': 'ineq', 'fun': lambda weights: 0.85 - np.sum(weights)},  # Sum of weights >= 0.85
        {'type': 'ineq', 'fun': lambda weights: np.sum(weights) - 1.0}  # Sum of weights <= 1
    )
bounds = tuple((0, 1) for x in range(n_stocks))


optimized = opt.minimize(objective_function, initial_weights, args= (returns), bounds=bounds, constraints=constraints)  # Adjust the method as needed
optimal_weights = optimized.x

Optimized Weights: [9.82333607e-13 5.39061979e-20 1.38369390e-12 3.86140681e-20
 2.29069602e-20 1.50190209e-12 3.84233936e-20 1.54462125e-12
 3.25768711e-20 4.52390859e-20 2.37245464e-20 4.92081415e-20
 2.68979731e-20 1.37077609e-12 6.45974883e-20 5.36032240e-20
 5.11060756e-12 4.42162817e-20 5.01258128e-20 5.14036290e-20
 5.68350680e-20 4.35903058e-20 3.76023617e-20 3.01175630e-20
 5.43683760e-20 8.66947871e-14 4.70738357e-20 3.79827102e-20
 2.00378940e-20 3.21956113e-20 4.52953939e-20 3.90423507e-20
 2.64354643e-20 1.38581833e-12 5.53811707e-03 5.01633838e-20
 6.00072337e-20 4.79308012e-20 9.81745870e-14 3.83279749e-13
 2.42892540e-20 2.66328642e-13 4.36613234e-20 3.19924095e-20
 2.77793320e-02 4.03867988e-20 4.85454577e-02 4.58096675e-20
 2.13622630e-20 1.70528748e-12 5.03085934e-20 4.74827227e-13
 2.43745610e-20 1.59748893e-12 4.68554257e-20 4.91533552e-20
 1.55642269e-12 4.12767706e-20 4.59243347e-20 5.45903177e-13
 3.44813229e-20 5.77985072e-20 2.12836895e-20 4.61093225e-20
 4.43

In [99]:
def calc_normalized_weights(optimal_weights, stock_names, threshold = 0.01):

    thresholded_weights = np.where(optimal_weights >= threshold, optimal_weights, 0)

    if np.sum(thresholded_weights) > 0:  # Prevent division by zero
        normalized_weights = thresholded_weights / np.sum(thresholded_weights)
    else:
        normalized_weights = thresholded_weights  # In case all are zero, which should not happen
    
    stocks_with_weights = [(stock, weight) for stock, weight in zip(stock_names, normalized_weights) if weight >= threshold]
    # Print stocks with their corresponding weights
    
    for stock, weight in stocks_with_weights:
        print(f"{stock}: {weight}")
    return normalized_weights

stock_names = returns.columns
normalized_weights = calc_normalized_weights(optimal_weights, stock_names)

ATVI: 0.028021700706215857
AVGO: 0.048969006359319014
CBOE: 0.2151005390019007
CHD: 0.03638581772687728
CME: 0.13053194160937764
CMG: 0.06101235294191748
COR: 0.10885409139976551
CRM: 0.023557214790272625
FDX: 0.07658190624843678
FICO: 0.07242754593603655
GE: 0.19855788327988066
